In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


**Setup**

Si la instalación de lightgbm da error, ejecutar en Anaconda Prompt 

`conda install -c conda-forge xgboost=1.4.0`

`conda install -c conda-forge lightgbm=3.2.1`

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Boosting en Scikit Learn

## 1. Introducción
En esta práctica vamos a comparar el rendimiento de los siguientes algoritmos:

- AdaBoostClassifier()
- GradientBoostingClassifier()


Para ello vamos a comenzar con la lectura del dataset de aceptabilidad de autos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
df = pd.read_csv('../Data/car.csv') 
df.info()

In [ ]:
df.head()

In [ ]:
display(df.doors.unique())
display(df.persons.unique())
display(df.acceptability.unique())

Esta vez vamos a codificar los atributos usando un esquema One Hot, es decir, los consideraremos como variables categóricas. También vamos a codificar el target usando el `LabelEncoder`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()
lab_enc.fit(df['acceptability'])

In [ ]:
y = lab_enc.transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1),drop_first=True)


In [ ]:
# vemos la forma final la matriz de features
print(X.shape)
X.head()

In [ ]:
# vemos la forma final del vector target
print(y.shape)
y

### Hacemos el split entre train y test sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

Para que los resultados sean consistentes hay que exponer los modelos exactamente al mismo esquema de validación cruzada.

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

## 2. Comparando la performance de los árboles de decisión y boosting
 
Ahora vamos a inicializar el clasificador de árbol de decisión, evaluar su rendimiento y compararlo con la perfomance de los modelos de boosting. Para ello, vamos a usar los siguientes métodos:

Para comparar los diferentes algoritmos armamos la función `evaluar_rendimiento`, que toma como input un estimador y un string con el nombre que le queramos poner, y ejecuta un `cross_val_score`

In [ ]:
def evaluar_rendimiento(modelo, nombre, X, y, cv):
    s = cross_val_score(modelo, X, y, cv=cv, n_jobs=-1)
    print("Rendimiento de {}:\t{:0.3} ± {:0.3}".format( \
        nombre, s.mean().round(3), s.std().round(3)))

Evaluamos el rendimiento de un árbol de Clasificación simple.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=1)

evaluar_rendimiento(dt,"Árbol de decisión", X_train, y_train, cv)

Probemos ahora los modelos de *boosting* AdaBoost y GradientBoosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

ab = AdaBoostClassifier(base_estimator=dt, n_estimators=500,random_state=1)
evaluar_rendimiento(ab, "AdaBoostClassifier",  X_train, y_train, cv)

In [ ]:
gb = GradientBoostingClassifier()
evaluar_rendimiento(gb, "GradientBoostingClassifier", X_train, y_train, cv)

Podemos ver que AdaBoost performa peor. Podríamos tratar de tunear los hiperparámetros para hacerlo funcionar mejor. 

In [ ]:
from sklearn.model_selection import GridSearchCV

params_ab = {"n_estimators": [500,1000],
          "learning_rate":[0.01, 0.1],
        "base_estimator__max_depth": [1, 2, 3]}

grid_ab = GridSearchCV(AdaBoostClassifier(base_estimator=dt,random_state=1), 
                       param_grid=params_ab, cv=cv, verbose=1, n_jobs=3)


In [ ]:
grid_ab.fit(X_train, y_train)

In [ ]:
grid_ab.best_estimator_

In [ ]:
evaluar_rendimiento(grid_ab.best_estimator_,  "AdaBoostClassifier + GS", X_train, y_train, cv)

No hemos logrado mejorar la performance del modelo Adaboost luego de intentar optimizar sus parametros.  
Veamos como nos va con Gradient Boosting

In [ ]:
params_gb = {'n_estimators':[500, 1000] , 
             'learning_rate':[0.001, 0.001, 0.1],
            'max_depth' : [1, 2, 3, 4]}

grid_gb = GridSearchCV(gb, param_grid=params_gb, cv=cv, verbose=1, n_jobs=3)

In [ ]:
grid_gb.fit(X_train, y_train)

In [ ]:
grid_gb.best_estimator_

In [ ]:
evaluar_rendimiento(grid_gb.best_estimator_, "GradientBoostingClassifier + GS", X_train, y_train, cv)

En este caso si vemos una mejora en la performance del Modelo de Gradient Boosting.  
Veamos a continuacion el valor de AUC y el gráfico de ROC

In [ ]:
from sklearn.metrics import roc_auc_score
xgb_auc = roc_auc_score(y_test,grid_gb.predict_proba(X_test),multi_class="ovr")
print("El valor del AUC es: ", xgb_auc)

In [ ]:
from scikitplot.metrics import plot_roc
plot_roc(y_test,grid_gb.predict_proba(X_test))

Finalmente vamos a graficar la importancia relativa de los feature para la predicción:

In [ ]:
model = grid_gb.best_estimator_
model

In [ ]:
importances = model.feature_importances_
importances

In [ ]:
# Recordamos que argsort nos devuelve un vector con indices del vector original 
# tal que este quede reordenado de mayor a menor. 
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html

# creamos una variable que tenga los indices indicando los valores de mayor a menor
indices = np.argsort(importances)[::-1]

# con dicha variable realizamos fancy indexing de manera de ordenar los labels del eje x.
names = X.columns[indices]

# Creamos el plot
plt.figure()

# Creamos plot title
plt.title("Feature Importance")

# Agregamos las barras
plt.bar(range(X.shape[1]), importances[indices])

# Agregamos los feature names 
plt.xticks(range(X.shape[1]), names, rotation=90)

# Show plot
plt.show()